In [1]:
import datadotworld as dw
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from vega_datasets import data
import folium
from folium.plugins import HeatMap
from IPython.display import HTML, display

In [21]:
# load data from data.world
# tutorial here: https://pypi.org/project/datadotworld/ 

results = dw.query('makeovermonday/london-cycle-hire-usage', # enter url end of dataset
                   'SELECT startstation_lat, startstation_lon, duration, startdate, enddate FROM tfl_cycle_hire_2017 LIMIT 300000') # add SQL to query data

df = results.dataframe

#df2 = df[['latitude','longitude']]

df.head()

,startstation_lat,startstation_lon,duration,startdate,enddate
0,51.490163,-0.190393,180,2017-04-05 22:47:00,2017-04-05 22:50:00
1,51.490163,-0.190393,180,2017-04-12 21:41:00,2017-04-12 21:44:00
2,51.490163,-0.190393,2040,2017-03-25 15:39:00,2017-03-25 16:13:00
3,51.514441,-0.087587,1140,2017-09-20 17:34:00,2017-09-20 17:53:00
4,51.490163,-0.190393,540,2017-06-19 08:43:00,2017-06-19 08:52:00


In [22]:
df.startdate = pd.to_datetime(df.startdate, format='%Y-%m-%d %H:%M:%S')
df.enddate = pd.to_datetime(df.enddate, format='%Y-%m-%d %H:%M:%S')

df['month'] = df.startdate.apply(lambda x: x.month)
df['week'] = df.startdate.apply(lambda x: x.week)
df['day'] = df.startdate.apply(lambda x: x.day)
df['hour'] = df.startdate.apply(lambda x: x.hour)

In [9]:
df.head()

,startstation_lat,startstation_lon,duration,startdate,enddate,month,week,day,hour
0,51.490163,-0.190393,180,2017-04-05 22:47:00,2017-04-05 22:50:00,4,14,5,22
1,51.490163,-0.190393,180,2017-04-12 21:41:00,2017-04-12 21:44:00,4,15,12,21
2,51.490163,-0.190393,2040,2017-03-25 15:39:00,2017-03-25 16:13:00,3,12,25,15
3,51.514441,-0.087587,1140,2017-09-20 17:34:00,2017-09-20 17:53:00,9,38,20,17
4,51.490163,-0.190393,540,2017-06-19 08:43:00,2017-06-19 08:52:00,6,25,19,8


In [24]:
def generateBaseMap(default_location=[51.5074, -0.1278], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, tiles="CartoDB dark_matter")
    return base_map

base_map = generateBaseMap()

base_map

In [25]:
HeatMap(data=df[['startstation_lat', 'startstation_lon', 'duration']].groupby(['startstation_lat', 'startstation_lon']).count().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map

In [26]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))

In [27]:
from folium.plugins import HeatMapWithTime

In [28]:
df_hour_list = []
for hour in df.hour.sort_values().unique():
    df_hour_list.append(df.loc[df.hour == hour, ['startstation_lat', 'startstation_lon', 'duration']].groupby(['startstation_lat', 'startstation_lon']).count().reset_index().values.tolist())


In [29]:
base_map = generateBaseMap(default_zoom_start=11)

HeatMapWithTime(df_hour_list, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)

base_map